# SUBMISSION GENERATOR

## 1-IMPORTS

In [1]:
import json
import os

## 2-DEFINING USEFUL FUNCTIONS

In [2]:
########## HEADER ##########
# Config file description :

# network_learning_rate : float : No idea, but certainly linked to the back propagation ? Default : 0.0005.

# network_n_classes : int : number of labels in the output. Default : 2.

# network_dropout : float : between 0 and 1 : percentage of neurons we want to keep. Default : 0.75.

# network_depth : int : number of layers WARNING : factualy, there will be 2*network_depth layers. Default : 6.

# network_convolution_per_layer : list of int, length = network_depth : number of convolution per layer. Default : [1 for i in range(network_depth)].

# network_size_of_convolutions_per_layer : list of lists of int [number of layers[number_of_convolutions_per_layer]] : Describe the size of each convolution filter.
# Default : [[3 for k in range(network_convolution_per_layer[i])] for i in range(network_depth)].

# network_features_per_layer : list of lists of int [number of layers[number_of_convolutions_per_layer[2]] : Numer of different filters that are going to be used.
# Default : [[64 for k in range(network_convolution_per_layer[i])] for i in range(network_depth)]. WARNING ! network_features_per_layer[k][1] = network_features_per_layer[k+1][0].

# network_trainingset : string : describe the trainingset for the network.

# network_downsampling : string 'maxpooling' or 'convolution' : the downsampling method.

# network_thresholds : list of float in [0,1] : the thresholds for the ground truthes labels.

# network_weighted_cost : boolean : whether we use weighted cost for training or not.
###########################

In [3]:
def default_config():
    """Generate the default config dict"""
    network_learning_rate = 0.0005
    network_n_classes = 3
    dropout = 0.75
    network_depth = 6
    network_convolution_per_layer = [3 for i in range(network_depth)]
    network_size_of_convolutions_per_layer = [[5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 5, 5]]  
    network_features_per_convolution =  [[[1, 10], [10, 20], [20, 30]], [[30, 40], [40, 50], [50, 60]],
                                        [[60, 70], [70, 80], [80, 90]], [[90, 100], [100, 110], [110, 120]],
                                        [[120, 130], [130, 140], [140, 150]], [[150, 160], [160, 170], [170, 180]]]
    trainingset = 'simulation_png'
    downsampling = 'maxpooling'
    thresholds = [0, 0.1, 0.8]
    weighted_cost = False
    batch_size = 1

    config = {
        'network_learning_rate': network_learning_rate,
        'network_n_classes': network_n_classes,
        'network_dropout': dropout,
        'network_depth': network_depth,
        'network_convolution_per_layer': network_convolution_per_layer,
        'network_size_of_convolutions_per_layer': network_size_of_convolutions_per_layer,
        'network_features_per_convolution': network_features_per_convolution,
        'network_trainingset': trainingset,
        'network_downsampling': downsampling,
        'network_thresholds': thresholds,
        'network_weighted_cost': weighted_cost,
        'network_batch_size': batch_size
    }
    return config

In [4]:
def generate_features(depth,network_first_num_features,features_augmentation,network_convolution_per_layer):

    increment = int(float(features_augmentation[1:]))

    if str(features_augmentation[0]) == 'p':
        # Add N features at each convolution layer.
        first_conv = [[1,network_first_num_features]]
        temp = [[network_first_num_features+i*increment,network_first_num_features+(i+1)*increment] 
                                for i in range(network_convolution_per_layer[0])[1:]]
        first_layer = first_conv + temp
        last_layer = first_layer
        network_features_per_convolution = [first_layer]

        for cur_depth in range(depth)[1:]:

            first_conv = [[last_layer[-1][-1],last_layer[-1][-1]+increment]]
            temp = [[last_layer[-1][-1]+i*increment,last_layer[-1][-1]+(i+1)*increment] for i in range(network_convolution_per_layer[cur_depth])[1:]]
            current_layer = first_conv+temp
            network_features_per_convolution = network_features_per_convolution + [current_layer]

            last_layer = current_layer

    elif str(features_augmentation[0]) == 'x':
        # Multiply the number of features by N at each "big layer".
        
        first_conv = [[1,network_first_num_features]]
        temp = [[network_first_num_features,network_first_num_features] 
                                for i in range(network_convolution_per_layer[0]-1)]
        first_layer = first_conv + temp
        last_layer = first_layer
        network_features_per_convolution = [first_layer]
        for cur_depth in range(depth)[1:]:
            first_conv = [[last_layer[-1][-1],last_layer[-1][-1]*increment]]
            temp = [[last_layer[-1][-1]*increment,last_layer[-1][-1]*increment] for i in range(network_convolution_per_layer[cur_depth]-1)]
            current_layer = first_conv+temp
            network_features_per_convolution = network_features_per_convolution + [current_layer]

            last_layer = current_layer

    else:
        raise 'Invalid input : please for features_augmentation' 
                                                 

    return network_features_per_convolution

In [5]:
def generate_dict(  network_learning_rate = 0.0005,
                    network_n_classes = 3,
                    dropout = 0.75,
                    network_depth = 6,
                    convolution_per_layer = 3,
                    network_size_of_convolutions = 3,  
                    features_augmentation = 'p10',
                    network_first_num_features = 16,
                    trainingset = 'simulation_png',
                    downsampling = 'maxpooling',
                    thresholds = [0, 0.1, 0.8],
                    weighted_cost = False,
                    batch_size = 1):

    """ features_augmentation : string : first caracter p for addition or x for multiplication. Rest of the string : imcrementation value. """

    network_learning_rate = network_learning_rate
    network_n_classes = network_n_classes
    dropout = dropout
    network_depth = network_depth
    
    trainingset = trainingset
    downsampling = downsampling
    thresholds = thresholds
    weighted_cost = weighted_cost

    network_first_num_features = network_first_num_features
    network_convolution_per_layer = [convolution_per_layer for i in range(network_depth)]
    network_size_of_convolutions_per_layer = [[network_size_of_convolutions for k in range(network_convolution_per_layer[i])] for i in range(network_depth)]

    network_features_per_convolution = generate_features(network_depth,network_first_num_features,features_augmentation,network_convolution_per_layer)

    config = {
        'network_learning_rate': network_learning_rate,
        'network_n_classes': network_n_classes,
        'network_dropout': dropout,
        'network_depth': network_depth,
        'network_convolution_per_layer': network_convolution_per_layer,
        'network_size_of_convolutions_per_layer': network_size_of_convolutions_per_layer,
        'network_features_per_convolution': network_features_per_convolution,
        'network_trainingset': trainingset,
        'network_downsampling': downsampling,
        'network_thresholds': thresholds,
        'network_weighted_cost': weighted_cost,
        'network_batch_size': batch_size
    }

    return config

In [6]:
def generate_heliosjob(bashname,configfile,config,path_trainingset,path_model,path_model_init = None):
    """Generate config file given a config dict. Generate the corresponding submission."""

    if not os.path.exists(path_model):
        os.makedirs(path_model)

    with open(os.path.join(path_model, configfile), 'w') as f:
        json.dump(config, f, indent=2)
        
    name_model = path_model.split('/')[-2]

    file = open(os.path.join(path_model, bashname),"w")
    file.write("#!/bin/bash \n")
    file.write("#PBS -N mapping_test \n")
    file.write("#PBS -A rrp-355-aa \n")
    file.write("#PBS -l walltime=43200 \n")
    file.write("#PBS -l nodes=1:gpus=1 \n")
    file.write("cd $SCRATCH/maxime/axondeepseg/models/" + name_model + "/ \n")
    file.write("source /home/maxwab/tf11-py27/bin/activate \n")
    file.write("module load compilers/gcc/4.8.5 compilers/java/1.8 apps/buildtools compilers/swig apps/git apps/bazel/0.4.3 \n")
    file.write("module load cuda/7.5 \n")
    file.write("module load libs/cuDNN/5 \n")
    file.write("python ../../AxonDeepSeg/trainingforhelios.py -co ")
    file.write(str(configfile))
    file.write(" -t ") 
    file.write(str(path_trainingset))
    file.write(" -m ")
    file.write(str(path_model))
    if path_model_init:
        file.write(" -i ")
        file.write(str(path_model_init))
        
    file.close()

In [7]:
def generate_guilliminjob(bashname,configfile,config,path_trainingset,path_model,path_model_init = None):
    """Generate config file given a config dict. Generate the corresponding submission."""

    if not os.path.exists(path_model):
        os.makedirs(path_model)

    with open(os.path.join(path_model, configfile), 'w') as f:
        json.dump(config, f, indent=2)
        
    name_model = path_model.split('/')[-2]

    file = open(os.path.join(path_model, bashname),"w")
    file.write("#!/bin/bash \n")
    file.write("#PBS -N "+ name_model +" \n")
    file.write("#PBS -A rrp-355-aa \n")
    file.write("#PBS -l walltime=43200 \n")
    file.write("#PBS -l nodes=1:gpus=1 \n")
    file.write("cd /gs/project/rrp-355-aa/maxwab/axondeepseg/models/" + name_model + "/ \n")
    file.write("module load foss/2015b Python/2.7.12 \n")
    file.write("source ~/maxwab/tf11-py27/bin/activate \n")
    file.write("module load GCC/5.3.0-2.26 Bazel/0.4.4 CUDA/7.5.18 \n")
    file.write("module load Tensorflow/1.0.0-Python-2.7.12 \n")
    file.write("python ../../AxonDeepSeg/trainingforhelios.py -co ")
    file.write(str(configfile))
    file.write(" -t ") 
    file.write(str(path_trainingset))
    file.write(" -m ")
    file.write(str(path_model))
    if path_model_init:
        file.write(" -i ")
        file.write(str(path_model_init))
        
    file.close()

## 3-GENERATING THE FILES

Defining the name of the files and models, and environment variables.

**IMPORTANT TO CHANGE AT EACH GENERATION**

In [8]:
server = 'Guillimin' # Choose between Guillimin and Helios

# Warning: model names must finish by "/"
modelnames = ['cv_2c_nw_d4_3_b8_x2-16/', 'cv_2c_nw_d5_3_b8_x2-16/', 'cv_2c_nw_d4_3_b8_x2-32/', 'cv_2c_nw_d5_3_b8_x2-32/', 
              'cv_2c_nw_d4_3_b16_x2-32/', 'cv_2c_nw_d5_3_b16_x2-32/']

path_models = '../models/'
path_data = '../../../trainingsets/'
    

Defining some test config files

In [9]:
config1 = generate_dict( network_learning_rate = 0.0005,
	                     network_n_classes = 2,
	                     dropout = 0.75,
	                     network_depth = 4,
	                     convolution_per_layer = 3,
	                     network_size_of_convolutions = 3,  
	                     features_augmentation = 'x2',
	                     network_first_num_features = 16,
	                     trainingset = 'SEM_2classes_reduced',
	                     downsampling = 'convolution',
	                     thresholds = [0, 0.5],
	                     weighted_cost = False,
                         batch_size = 8)

config2 = generate_dict( network_learning_rate = 0.0005,
	                     network_n_classes = 2,
	                     dropout = 0.75,
	                     network_depth = 5,
	                     convolution_per_layer = 3,
	                     network_size_of_convolutions = 3,  
	                     features_augmentation = 'x2',
	                     network_first_num_features = 16,
	                     trainingset = 'SEM_2classes_reduced',
	                     downsampling = 'convolution',
	                     thresholds = [0, 0.5],
	                     weighted_cost = False,
                         batch_size = 8)

config3 = generate_dict( network_learning_rate = 0.0005,
	                     network_n_classes = 2,
	                     dropout = 0.75,
	                     network_depth = 4,
	                     convolution_per_layer = 3,
	                     network_size_of_convolutions = 3,  
	                     features_augmentation = 'x2',
	                     network_first_num_features = 32,
	                     trainingset = 'SEM_2classes_reduced',
	                     downsampling = 'convolution',
	                     thresholds = [0, 0.5],
	                     weighted_cost = False,
                         batch_size = 8)

config4 = generate_dict( network_learning_rate = 0.0005,
	                     network_n_classes = 2,
	                     dropout = 0.75,
	                     network_depth = 5,
	                     convolution_per_layer = 3,
	                     network_size_of_convolutions = 3,  
	                     features_augmentation = 'x2',
	                     network_first_num_features = 32,
	                     trainingset = 'SEM_2classes_reduced',
	                     downsampling = 'convolution',
	                     thresholds = [0, 0.5],
	                     weighted_cost = False,
                         batch_size = 8)

config5 = generate_dict( network_learning_rate = 0.0005,
	                     network_n_classes = 2,
	                     dropout = 0.75,
	                     network_depth = 4,
	                     convolution_per_layer = 3,
	                     network_size_of_convolutions = 3,  
	                     features_augmentation = 'x2',
	                     network_first_num_features = 32,
	                     trainingset = 'SEM_2classes_reduced',
	                     downsampling = 'convolution',
	                     thresholds = [0, 0.5],
	                     weighted_cost = False,
                         batch_size = 16)

config6 = generate_dict( network_learning_rate = 0.0005,
	                     network_n_classes = 2,
	                     dropout = 0.75,
	                     network_depth = 5,
	                     convolution_per_layer = 3,
	                     network_size_of_convolutions = 3,  
	                     features_augmentation = 'x2',
	                     network_first_num_features = 32,
	                     trainingset = 'SEM_2classes_reduced',
	                     downsampling = 'convolution',
	                     thresholds = [0, 0.5],
	                     weighted_cost = False,
                         batch_size = 16)


We now generate the bash file that will launch all submissions at once, as well as the required sh files.

In [10]:
config_list = [config1,config2,config3,config4,config5,config6]
file = open(path_models + "/global_submission.sh","w") # We create the general submission file at the root of the models folder

for i,config in enumerate(config_list):

    submission_filename = "submission"+str(i)+".sh"
    if server == 'Helios':
        file.write('msub ')
    elif server == 'Guillimin':
        file.write('qsub ')
    file.write(str(modelnames[i])+'/')
    file.write(str(submission_filename))
    if server == 'Guillimin':
        file.write(' -l nodes=1:gpus=1:exclusive_process') # To avoid a server error. Check that the number of nodes is the good one !
    file.write('\n')
    
    if server == 'Helios':
        generate_heliosjob(submission_filename, 'config_network.json', config,
                       path_trainingset = path_data+config['network_trainingset']+'/training/',
                       path_model = path_models+modelnames[i]
                      )
    elif server == 'Guillimin':
        generate_guilliminjob(submission_filename, 'config_network.json', config,
                       path_trainingset = path_data+config['network_trainingset']+'/training/',
                       path_model = path_models+modelnames[i]
                      )

file.close()